# Property Rental Value Estimator - a QuAM

## Background

In [19]:
# imports
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridBox, Layout, Label
import pandas as pd
from sklearn.linear_model import LinearRegression
from functools import reduce

In [28]:
# load data and train model
data = pd.read_csv("data_clean.csv")
features = data.iloc[:, :-1]
targets = data.iloc[:, -1]
regressor = LinearRegression()  
regressor.fit(features, targets)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
# initialize variables
Bedrooms_options = features["Bedrooms"].unique()
Bathrooms_options = features["Bathrooms"].unique()
Amenities_options = ['Balcony',
                     'Barbecue Area',
                     'Built in Wardrobes',
                     'Central A/C',
                     "Children's Play Area",
                     "Children's Pool",
                     'Concierge',
                     'Covered Parking',
                     'Kitchen Appliances',
                     'Lobby in Building',
                     'MD Properties',
                     'Maid Service',
                     'Maids Room',
                     'Networked',
                     'Pets Allowed',
                     'Private Garden',
                     'Private Gym',
                     'Private Jacuzzi',
                     'Private Pool',
                     'Security',
                     'Shared Gym',
                     'Shared Pool',
                     'Shared Spa',
                     'Study',
                     'View of Landmark',
                     'View of Water',
                     'Walk-in Closet']
Type_options = ['Apartment', 'Villa', 'Hotel Apartments', 'Compound', 'Penthouse', 'Townhouse', 'Duplex']
Type_options.sort()
Furnishing_options = ['Furnished', 'Partly furnished', 'Unfurnished']
Location_options = [ 'The Pearl',
                     'West Bay',
                     'Al Sadd',
                     'Mesaimeer',
                     'Ain Khaled',
                     'Umm Salal Ali',
                     'Salata',
                     'Al Gharrafa',
                     'West Bay Lagoon',
                     'Al Ghanim',
                     'Al Thumama',
                     'Musheireb',
                     'Izghawa',
                     'Old Airport Road',
                     'Fereej Abdul Aziz',
                     'Corniche Road',
                     'Al Aziziyah',
                     'Umm Ghuwailina',
                     'Al Waab',
                     'Al Muntazah',
                     'Al Mirqab',
                     'Diplomats Area',
                     'Al Sakhama',
                     'Muaither Area',
                     'Al Dafna',
                     'Al Mirqab Al Jadeed',
                     'Al Mansoura',
                     'Mughalina',
                     'Najma',
                     'Al Nasr',
                     'Al Maamoura',
                     'Doha Al Jadeed',
                     'Fereej Bin Mahmoud',
                     'Hazm Al Markhiya',
                     'Al Jasra',
                     'Fereej Bin Omran',
                     'Abu Hamour',
                     'Al Rayyan',
                     'Al Hilal',
                     'Al Najada',
                     'Rawdat Al Matar',
                     'Jeliah',
                     'Al Markhiya',
                     'Salwa Road',
                     'Umm Salal Mohammad',
                     'Al Nuaija',
                     'Al Duhail',
                     'Al Messila',
                     'Umm Al Seneem',
                     'Madinat Khalifa',
                     'Fereej Abdel Aziz',
                     'Al Rawda',
                     'Al Hitmi',
                     'Airport Area',
                     'Fereej Kulaib',
                     'Msheireb Downtown Doha',
                     'Barwa City',
                     'AlMuraikh',
                     'New Doha',
                     'Al Luqta',
                     'D-Ring',
                     'E-Ring',
                     'Al Soudan',
                     'B-Ring Road',
                     'Business District',
                     'Ras Abu Aboud',
                     'Wadi Al Sail',
                     'Fereej Al Ali',
                     'Esghawa',
                     'Al Jebailat']
Location_options.sort()

### Pre-processing

In [22]:
# prepare widgets
beds = widgets.IntSlider(
    min=Bedrooms_options[0],
    max=Bedrooms_options[-1],
    step=1,
    description='Bedrooms:')

baths = widgets.IntSlider(
    min=Bathrooms_options[0],
    max=Bathrooms_options[-1],
    step=1,
    description='Bathrooms:')

size = widgets.IntText(
    description='Size (sqm):',
    value=125)

all_amenities = []
for am in Amenities_options:
    w = widgets.Checkbox(description=am, layout=Layout(width='auto', height='auto'))
    all_amenities.append(w)
am_grid = GridBox(
            children=all_amenities,
            layout=Layout(
                width='100%',
                grid_template_columns='250px 250px 250px',
                grid_template_rows='25px 25px 25px'))
am_label = Label("Select your amenities:")
        
locations = widgets.Dropdown(
    options=Location_options,
    value='The Pearl',
    description='Location:')

furnishings = widgets.RadioButtons(
        options=Furnishing_options,
        description='Furnishing:')

types = widgets.Dropdown(
    options=Type_options,
    value='Apartment',
    description='Type:')


### Post-processing

In [23]:
def get_prediction():
    #making entry
    entry = [size.value, beds.value, baths.value]
    
    am_bools = list(map(lambda b: b.value, all_amenities))
    nr_of_amenities = reduce((lambda x, y: x + y), am_bools)
    entry.append(nr_of_amenities)
    
    am_needed = ['Balcony', 'Built in Wardrobes', 'Maids Room', 'Private Garden']
    amenities_needed = [am.value for am in all_amenities if am.description in am_needed]
    amenities_needed_bool = list(map(lambda b: 1 if b else 0, amenities_needed))
    entry += amenities_needed_bool
    
    types_bool = list(map(lambda t: 1 if t == types.value else 0, Type_options))
    entry += types_bool
    
    furnishings_bool = list(map(lambda f: 1 if f == furnishings.value else 0, Furnishing_options))
    entry += furnishings_bool
    
    locations_bool = list(map(lambda l: 1 if l == locations.value else 0, Location_options))
    entry += locations_bool
    
    #making prediction
    prediction = regressor.predict(pd.DataFrame([entry]))
    
    return(prediction[0])

In [24]:
button = widgets.Button(description="Submit")
output = widgets.Output()

def submitted(b):
    pred = get_prediction()
    with output:
        print("Your property's value is estimated at:", pred)

button.on_click(submitted)

## QuAM

Please answer these following questions accurately and click the submit button so we can give you a precise estimation for the value of your property.

In [25]:
display(types, size, beds, baths, furnishings, locations)

Dropdown(description='Type:', options=('Apartment', 'Compound', 'Duplex', 'Hotel Apartments', 'Penthouse', 'To…

IntText(value=125, description='Size (sqm):')

IntSlider(value=1, description='Bedrooms:', max=9, min=1)

IntSlider(value=1, description='Bathrooms:', max=8, min=1)

RadioButtons(description='Furnishing:', options=('Furnished', 'Partly furnished', 'Unfurnished'), value='Furni…

Dropdown(description='Location:', index=62, options=('Abu Hamour', 'Ain Khaled', 'Airport Area', 'Al Aziziyah'…

In [26]:
display(am_label, am_grid)

Label(value='Select your amenities:')

GridBox(children=(Checkbox(value=False, description='Balcony', layout=Layout(height='auto', width='auto')), Ch…

In [27]:
display(button, output)

Button(description='Submit', style=ButtonStyle())

Output()